### Detecting the cars in the video using OpenCV Library

In [1]:
import cv2

file = "traffic1.avi"
cap = cv2.VideoCapture(file)

# Logic of reference frame
# if we can subtract the frame without cars(first frame in this case) from frame with cars, then we will be left with frame with cars alone
ref_frame = None
# define image area
Image_area = None

while True:
    ret,frame = cap.read()
    # for every frame available in the video, ret variable returns TRUE
    # but at the end of video, when there is no next frame avaialble
    # then the ret returns FALSE and we will get an error
    # so we are writing if condition as below, in this case, when ret is FALSE
    # the video window closes automatically with no error
    if ret is False:
        break
    else:
        if ref_frame is None:
            ref_frame = frame
            ref_frame =cv2.cvtColor(ref_frame,cv2.COLOR_BGR2GRAY)
            Image_area = ref_frame.shape[0]*ref_frame.shape[1]
            continue    
        
        #step -1 : convert to gray scale
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        #step -2 : apply reference frame logic
        difference = cv2.absdiff(ref_frame,gray)
        # use filters to remove the noise to have even better thresh frame
        blur = cv2.medianBlur(difference,31)
        
        # step -3 : Set the Threshold
        # the threshold is not that good, let;s try removing OTSU and 
        # giving manual threh value
        #f, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        f, thresh = cv2.threshold(blur,20,255,cv2.THRESH_BINARY)
        
        #step -4 : Finding contours
        (_,contours,_) = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        #print(len(contours))
        
        # step -5 : Draw the contours
        # contours should be drawn on the original frames
        #cv2.drawContours(frame,contours,-1,(0,0,255),2)
        
        #step -6 : Let's draw bounding boxes to make it look more specific
        for i in contours:
            # step -7 : Remove False Contours by using AREA
            contour_area = cv2.contourArea(i)
            if (contour_area > 0.001*Image_area) and (contour_area < 0.1*Image_area):
                (x,y,w,h) = cv2.boundingRect(i)
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        
        
        #resize the window
        cv2.namedWindow('Video',cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Video",600,600)
        #visualize threshold frames
        #cv2.imshow("Video",thresh)
        #visualize original frames
        cv2.imshow("Video",frame)
    
        keypress = cv2.waitKey(1)
        if keypress == ord('q'):
            break   
    
cv2.destroyAllWindows()